In [1]:
import io
import requests

import numpy as np
import pandas as pd

import boto3

In [2]:
from PIL import Image

In [3]:
from tqdm.notebook import tqdm

In [4]:
s3 = boto3.client("s3")

In [5]:
df = pd.read_csv("data/photos.tsv000",sep="\t")
df.head()

,photo_id,photo_url,photo_image_url,photo_submitted_at,photo_featured,photo_width,photo_height,photo_aspect_ratio,photo_description,photographer_username,...,photo_location_longitude,photo_location_country,photo_location_city,stats_views,stats_downloads,ai_description,ai_primary_landmark_name,ai_primary_landmark_latitude,ai_primary_landmark_longitude,ai_primary_landmark_confidence
0,2Q8zDWkj0Yw,https://unsplash.com/photos/2Q8zDWkj0Yw,https://images.unsplash.com/photo-141520117961...,2014-11-05 15:26:26.678711,t,4192,2794,1.50,NaN,lanceanderson,...,NaN,NaN,NaN,15854580,146388,school of jellyfish swimming in body of water,NaN,NaN,NaN,NaN
1,tsBDNuCJiLg,https://unsplash.com/photos/tsBDNuCJiLg,https://images.unsplash.com/photo-141768928330...,2014-12-04 10:31:44.647012,t,4324,2880,1.50,Surfer in the ocean,thehungryjpeg,...,NaN,NaN,NaN,3157825,8247,surfer walking on body of water during daytime,NaN,NaN,NaN,NaN
2,A93gsuMxVcE,https://unsplash.com/photos/A93gsuMxVcE,https://images.unsplash.com/photo-142981401899...,2015-04-23 18:33:44.024841,t,2000,1333,1.50,Barren Countryside,jeremydgreat,...,NaN,NaN,NaN,2159829,7064,landscape photo of road beside field of grass,NaN,NaN,NaN,NaN
3,oYIdH6bFssk,https://unsplash.com/photos/oYIdH6bFssk,https://images.unsplash.com/photo-143275722183...,2015-05-27 20:07:28.35245,t,4288,2848,1.51,Lightbulb reflections,chelaxydp,...,NaN,NaN,NaN,2861489,11778,closed up photo of orange lightened lamp,NaN,NaN,NaN,NaN
4,wgLPy2YBXuc,https://unsplash.com/photos/wgLPy2YBXuc,https://images.unsplash.com/photo-143205996405...,2015-05-19 18:26:15.223161,t,5312,2988,1.78,NaN,csogi,...,NaN,NaN,NaN,6037626,52263,white clouds and blue sky at daytime,NaN,NaN,NaN,NaN


In [6]:
# for img_id, url in tqdm(df[["photo_id","photo_image_url"]].values,desc="Images Downloaded"):
#     resp = requests.get(url)
#     b = io.BytesIO(resp.content)
#     b.seek(0)
#     s3.upload_fileobj(b,"ap-unsplash-images",f"{img_id}.jpg")

In [7]:
def get_crop(img):
    w, h = img.size
    if w > h:
        d = (w - h) / 2
        return (0,-d,w,h+d)
    else:
        d = (h - w) / 2
        return (-d,0,w+d,h)
    
def transform_img(img,new_size=(960,960)):
    return img.crop(get_crop(img)).resize(new_size)

def drop_alpha(img):
    try: 
        return Image.fromarray(np.array(img)[:,:,:3])
    except: return img

def load_image(url: str):
    resp = requests.get(url)
    b = io.BytesIO(resp.content)
    return Image.open(b)

def upload_image(img, img_id: str):
    with io.BytesIO() as b:
        img.save(b,format="JPEG")
        b.seek(0)
        s3.upload_fileobj(b,"ap-unsplash-images",f"reshaped/{img_id}.jpg")
        
def process_image(url,img_id=None):
    if img_id is None:
        url, img_id = url
    img = load_image(url)
    img = drop_alpha(img)
    img = transform_img(img)
    upload_image(img, img_id)

In [8]:
with open("tmp/existing-files.txt") as f:
    existing_files = f.read().split("\n")

In [9]:
from multiprocessing import Pool

In [10]:
with Pool() as p:
    p.map(
        process_image,
        df[
            df.photo_id.apply(lambda pid: f"reshaped/{pid}.jpg" not in existing_files)
        ][["photo_image_url","photo_id"]].values
    )

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (121500000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (92160000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (98010000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/PIL/Image.py:2835: DecompressionBombWarning: Image size (99991727 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,
/home/ec2-user/anaconda3/envs/p

OSError: cannot write mode P as JPEG